## Throughputs Times


Below, the supermarket simulation code. It remains largely unchanged, except for the addition of the `all_customers` list, to which each Customer component is appended on init.


This list is used to loop through Customer components and their action logs, to extract throughput times for their supermarket visit and specific interactions:
- cart/basket queue
- bread queue
- cheese and dairy queue
- checkout queue


In [1]:
import salabim as sim

env = sim.Environment(time_unit='seconds')

# customer_to_log = 500

#Shopping baskets and carts:
carts = env.Resource('carts', capacity=45) #45
baskets = env.Resource('baskets', capacity=500)#several hundred
cart_basket_distribution = sim.Pdf((carts, 0.8,baskets, 0.2))
customer_basketcart_distribution_monitor = sim.Monitor("Customer basket and cart monitor")
#shopping categories:
# fruit_and_vegetables 
# meat_and_fish
# bread
# cheese_and_dairy
# canned_and_packed_food
# frozen_foods
# drinks
item_taking_distribution = sim.Uniform(20,30)

#Clerks
#Bread and cheese use resources as they are working via counters. Checkout likely has to use queue as the customers need to choose the smallest one
bread_clerks = env.Resource('bread_clerks', capacity=4) #4 employees, 1-6 items takes 2 min
bread_time_distribution = sim.Exponential(2*60)
cheese_and_dairy_clerks = env.Resource('bread_clerks', capacity=3) #3 employees, 1 min avg.
cheese_and_dairy_time_distribution = sim.Exponential(1*60)

#Checkouts
number_of_checkouts = 3
time_per_item_distribution = sim.Exponential(1.1)
payment_time_distribution = sim.Uniform(40, 60)

#Distributions of items per customer
fruit_and_vegetables_distribution = sim.Triangular(4, 22,10)  # min=4, mode=10, max=22
meat_and_fish_distribution = sim.Triangular(0, 9, 4)           # min=0, mode=4, max=9
bread_distribution = sim.Triangular(1, 10, 4)                  # min=1, mode=4, max=10
cheese_and_dairy_distribution = sim.Triangular(1, 11, 3)       # min=1, mode=3, max=11
canned_and_packed_food_distribution = sim.Triangular(6, 35, 17)# min=6, mode=17, max=35
frozen_foods_distribution = sim.Triangular(2, 19, 8)           # min=2, mode=8, max=19
drinks_distribution = sim.Triangular(1, 20, 9)                 # min=1, mode=9, max=20

#Route choice distribution

#Routes
route1 = [
"fruit_and_vegetables",
"meat_and_fish",
"bread",
"cheese_and_dairy",
"canned_and_packed_food",
"frozen_foods",
"drinks",
] #ABCDEF, 80% 

route2 = [
"meat_and_fish",
"bread",
"cheese_and_dairy",
"fruit_and_vegetables",
"canned_and_packed_food",
"frozen_foods",
"drinks",
] #BCDEFG, 20%

route_distribution = sim.Pdf((route1, 0.8, route2, 0.2))

### CHANGE HERE ###
all_customers = []

class Customer(sim.Component):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.route = route_distribution.sample()
        self.shopping_list = {
            "fruit_and_vegetables": int(round(fruit_and_vegetables_distribution.sample())),
            "meat_and_fish": int(round(meat_and_fish_distribution.sample())),
            "bread": int(round(bread_distribution.sample())),
            "cheese_and_dairy": int(round(cheese_and_dairy_distribution.sample())),
            "canned_and_packed_food": int(round(canned_and_packed_food_distribution.sample())),
            "frozen_foods": int(round(frozen_foods_distribution.sample())),
            "drinks": int(round(drinks_distribution.sample())),
        }
        self.progress = 0
        self.carrying = None
        self.actions_log = []
        
        all_customers.append(self)

    def log_action(self, action):
            """Helper function to log an action with the current time."""
            self.actions_log.append((env.now(), action))
        
    def process(self):
        """"
        Process determines what the customer will do. At the start they will take a cart or basket. Afterwards they will traverse their route and take the items they need according to their shopping list. If they have finished their route (when progress is equal to the length of the shopping list), they will go to the checkout.
        """
        self.start_shopping()
        for next_product in self.route:
            if self.shopping_list[next_product]>0:
                self.get_product(next_product)   
        self.go_to_checkout()
            
    def start_shopping(self):
        #Get shopping cart or basket
        want_to_carry = cart_basket_distribution.sample()
        self.log_action(f"Entered cart/basket queue for {want_to_carry}")
        self.request(want_to_carry)
        customer_basketcart_distribution_monitor.tally(want_to_carry)
        self.log_action(f"Got {want_to_carry}")
        self.carrying = want_to_carry
        
    def go_to_checkout(self):
        #enter emptiest queue
        emptiest_queue = min(checkouts, key=lambda checkout: checkout.requesters().length()) 
        self.log_action(f"Entered checkout queue {emptiest_queue}")
        self.request(emptiest_queue) 
        self.log_action(f"Started checking out")
        item_scan_time = sum(time_per_item_distribution.sample() for _ in range(sum(self.shopping_list.values())))
        self.hold(item_scan_time+payment_time_distribution.sample()) #hold the customer for scanning all items and during payment
        self.log_action(f"Finished checking out")
        #return cart/basket       
        #print log if we want to debug
        '''
        if customer_to_log:
            if self.name() == f"customer.{customer_to_log}":
                print(self.carrying.claimers().print_info())
                print(f"Customer's Action Log for customer {self.name()}:")
                for time, action in self.actions_log:
                    print(self)
                    print(f"At time {time}, customer: {action}")
        '''
        
    def get_product(self, product):
        #Move to product location #<- only needed in animation
        if product == "cheese_and_dairy":
            self.log_action(f"requesting cheese and dairy")
            self.request(cheese_and_dairy_clerks)
            self.log_action(f"Being helped for cheese and dairy")
            self.hold(cheese_and_dairy_distribution.sample())
            self.log_action(f"Got cheese and dairy")
            self.release(cheese_and_dairy_clerks)
        elif product == "bread":
            self.log_action(f"requesting bread")
            self.request(bread_clerks)
            self.log_action(f"Being helped for bread")
            self.hold(bread_time_distribution.sample())
            self.log_action(f"Got bread")
            self.release(bread_clerks)
        else:
            amount = self.shopping_list[product]
            self.log_action(f"Getting {product}")
            for _ in range(amount):
                self.hold(time_per_item_distribution.sample())
                self.log_action(f"Got {product}")

#customer generation
customer_distribution = [30, 80, 110, 90, 80, 70, 80, 90, 100, 120, 90, 40] #Expected total = 980
for index, customer_count  in enumerate(customer_distribution):
    env.ComponentGenerator(Customer, iat=env.Exponential(3600/customer_count), at=index*60*60, duration=60*60) #assumes time in seconds
    # print(customer_count)
customer_arrival_monitor = sim.Monitor("Customer arrival monitor")

checkouts = [] #dictionary to map checkouts to clerks
for i in range(number_of_checkouts):
    checkouts.append(env.Resource(f"checkout_clerk{i}", capacity = 1)) #3, 1.1s per item avg. payment 40-60s

env.run(duration=60*60*12)

---
### Average Throughput

The first and last actions of the action_log for any Customer are the 'Entered cart/basket queue' and 'Finished checking out'. Their respective timestamps represent the beginning and end of the Customer's visit to the supermarket, and thus the time passing between these timestamps is the throughput.

The current model implementation has a problem: there are customers that are "stuck" in the supermarket, because the 12 hours are up before they are checked out. These should not be considered, as their [-1] timestamps may not be the end of their shopping time.

In [2]:
# Initializing empty throughput list
throughput_times = []

# Loop over customers in the list
for customer in all_customers:
    # If the customer's last action is not 'Finished checking out', they aren't taken into account
    if customer.actions_log[-1][1] != 'Finished checking out':
        continue
    # Double indexing: first for the number of the action, second for the timestamp
    # If the second index is [1], the action name is selected (instead of the env.now() result)
    first_action_time = customer.actions_log[0][0]
    last_action_time = customer.actions_log[-1][0]
    # Throughput calculation and appending to main list
    throughput_time = last_action_time - first_action_time
    throughput_times.append(throughput_time)

average_throughput = sum(throughput_times) / len(throughput_times)

print(f'The average Customer visit lasted a total of {round(average_throughput, 2)} seconds, or {round(average_throughput/60, 2)} minutes.')

The average Customer visit lasted a total of 1009.28 seconds, or 16.82 minutes.


---
### Average Cart/Basket queue

For the initial queue throughput, a similar idea can be applied. The first and second actions in the log are always 'Entered cart/basket queue' and 'Got Resource' (either a cart or a basket). The average throughput can be calculated just like before.

In [3]:
cb_q_times = []

for customer in all_customers:
    
    enter_cb_q_time = customer.actions_log[0][0]
    leave_cb_q_time = customer.actions_log[1][0]

    cb_q_time = leave_cb_q_time - enter_cb_q_time
    cb_q_times.append(cb_q_time)

average_cb_q_time = sum(cb_q_times) / len(cb_q_times)

print(f'Customers waited for a basket/cart for an average of {round(average_cb_q_time, 2)} seconds, or {round(average_cb_q_time/60, 2)} minutes.')

Customers waited for a basket/cart for an average of 0.0 seconds, or 0.0 minutes.


##### This finding leads me to believe that no customers ever waited for either a cart or a basket. I don't know if this is realistic.

---
### Bread and Dairy queues

For these throughputs, the difference between the 'Requesting bread/dairy' action and the 'Got bread/dairy' is the throughput. The 'getting helped' action happens at the same env.now() instant as the request, so it isn't relevant.

In [4]:
bread_q_times = []

for customer in all_customers:
    for time, action in customer.actions_log:
        if action == 'requesting bread':
            bread_q_start = time
        if action == 'Got bread':
            bread_q_finish = time

    bread_q = bread_q_finish - bread_q_start
    bread_q_times.append(bread_q)

average_bread_q_time = sum(bread_q_times) / len(bread_q_times)
print(f'Customers waited for an average of {round(average_bread_q_time, 2)} seconds at bread clerk, or {round(average_bread_q_time/60, 2)} minutes.')

Customers waited for an average of 242.54 seconds at bread clerk, or 4.04 minutes.


In [5]:
dairy_q_times = []

for customer in all_customers:
    for time, action in customer.actions_log:
        if action == 'requesting cheese and dairy':
            dairy_q_start = time
        if action == 'Got cheese and dairy':
            dairy_q_finish = time

    dairy_q = dairy_q_finish - dairy_q_start
    dairy_q_times.append(dairy_q)

average_dairy_q_time = sum(dairy_q_times) / len(dairy_q_times)
print(f'Customers waited for an average of {round(average_dairy_q_time, 2)} seconds at cheese and dairy clerk, or {round(average_dairy_q_time/60, 2)} minutes.')

Customers waited for an average of 4.9 seconds at cheese and dairy clerk, or 0.08 minutes.


---
### Checkout queue

For the checkout queues, same principle as total throughput and carts/basket queue applies. The third to last action of every log is 'Entered checkout queue', followed by the number of the checkout chosen. The checkout throughput can either be from this action's timestamp to the second to last, 'Started checking out', or the final one, 'Finished checking out'. This depends on what the definition is.

The third to last action can also be used to gather clerk utilization data. The second to last character of the action string is the number of checkout used; the usage percentage of each checkout can be extracted this way.

In [6]:
checkout_q_times = []
checkout_utilization = {'0': 0, '1': 0, '2': 0}
total_checkouts = 0

for customer in all_customers:
    
    if customer.actions_log[-1][1] != 'Finished checking out':
        continue
    
    enter_checkout_q_time = customer.actions_log[-3][0]
    leave_checkout_q_time = customer.actions_log[-2][0]
    
    checkout_q_time = leave_checkout_q_time - enter_checkout_q_time
    checkout_q_times.append(checkout_q_time)
    
    # Triple index, this time pointing to the second to last character of the action string
    used_checkout = customer.actions_log[-3][1][-2]
    checkout_utilization[used_checkout] += 1
    total_checkouts += 1

average_checkout_q_time = sum(checkout_q_times) / len(checkout_q_times)

print(f'Customers waited at the checkout for an average of {round(average_checkout_q_time, 2)} seconds, or {round(average_checkout_q_time/60, 2)} minutes.\n')

for k,v in checkout_utilization.items():
    print(f'Checkout {k} was used {v} times, which is {round(v/total_checkouts*100, 2)}% of the total utilization.')

Customers waited at the checkout for an average of 577.39 seconds, or 9.62 minutes.

Checkout 0 was used 343 times, which is 35.65% of the total utilization.
Checkout 1 was used 321 times, which is 33.37% of the total utilization.
Checkout 2 was used 298 times, which is 30.98% of the total utilization.
